# Exploratory Data Analysis - Chest X-ray Dataset

Análise exploratória da base de dados [Chest X-ray Image (COVID19, PNEUMONIA, and NORMAL)](https://www.kaggle.com/datasets/alsaniipe/chest-x-ray-image) para o Trabalho 2 de Redes Neurais.

Este notebook irá:
1. Verificar a estrutura do dataset
2. Analisar a distribuição das classes
3. Examinar características das imagens (dimensões, formato, canais)
4. Visualizar exemplos de imagens de cada classe
5. Identificar possíveis problemas ou desafios no dataset

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import torch
from torchvision import transforms as T
from torchvision.transforms import v2
import random
from pathlib import Path
import plotly.express as px
from tqdm.notebook import tqdm

# Para visualizar as imagens no notebook
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Configuração e Download do Dataset

Primeiro, vamos configurar os caminhos para o dataset e verificar se ele já está disponível localmente ou se precisamos fazer o download.

In [1]:
# Importar a biblioteca para download do Kaggle
try:
    import kagglehub
    KAGGLE_AVAILABLE = True
except ImportError:
    print("kagglehub não está instalado. Tentando instalar...")
    !pip install kagglehub
    try:
        import kagglehub
        KAGGLE_AVAILABLE = True
    except ImportError:
        print("Falha ao instalar kagglehub. Dataset deverá estar disponível localmente.")
        KAGGLE_AVAILABLE = False

# Definir caminhos do dataset
BASE_PATH = Path('/home/luiscamara/RedesNeurais_202501/Trabalho2')
DATA_PATH = BASE_PATH / 'data'

# Verificar se o diretório de dados existe
if not DATA_PATH.exists():
    os.makedirs(DATA_PATH, exist_ok=True)
    
# Dataset ID do Kaggle
DATASET_ID = "alsaniipe/chest-x-ray-image"

# Função para download do dataset
def download_dataset():
    if KAGGLE_AVAILABLE:
        print("Iniciando download do dataset do Kaggle...")
        try:
            kagglehub.init()
            kagglehub.dataset_download(DATASET_ID, DATA_PATH)
            print(f"Dataset baixado com sucesso para {DATA_PATH}")
            return True
        except Exception as e:
            print(f"Erro ao baixar o dataset: {e}")
            return False
    else:
        print("kagglehub não está disponível. O download não será realizado.")
        return False

# Verificar se o dataset já está disponível localmente
dataset_dirs = ['chest_xray_train', 'chest_xray_test']
dataset_available = all((DATA_PATH / dir_name).exists() for dir_name in dataset_dirs)

if not dataset_available:
    print("Dataset não encontrado localmente. Tentando baixar...")
    download_success = download_dataset()
    if not download_success:
        print("Não foi possível baixar o dataset. Por favor, baixe manualmente do Kaggle e extraia para o diretório de dados.")
else:
    print(f"Dataset já está disponível em {DATA_PATH}")

# Configurar caminhos para os conjuntos de treino e teste
TRAIN_PATH = DATA_PATH / 'chest_xray_train'
TEST_PATH = DATA_PATH / 'chest_xray_test'

# Verificar se os diretórios existem
print(f"Diretório de treino existe: {TRAIN_PATH.exists()}")
print(f"Diretório de teste existe: {TEST_PATH.exists()}")

kagglehub não está instalado. Tentando instalar...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 KB 3.4 MB/s eta 0:00:00
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached requests-2.32.4-py3-none-any.whl (64 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (751 kB)
  Using cached idna-3.10-py3-none-any.whl (70 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.2/159.2 KB 13.9 MB/s eta 0:00:00
  Using cached charset_normalizer-3.4.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (149 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)


/home/luiscamara/RedesNeurais_202501/RedesNeurais/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'Path' is not defined

## 2. Análise da Estrutura do Dataset

Vamos examinar a estrutura do dataset, incluindo a quantidade de imagens em cada classe (COVID-19, Pneumonia, Normal) nos conjuntos de treino e teste.

In [ ]:
# Função para contar imagens por classe
def count_images_in_directory(directory):
    if not os.path.exists(directory):
        return 0
    return len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and 
               (f.endswith('.jpg') or f.endswith('.jpeg') or f.endswith('.png'))])

# Classes disponíveis no dataset
classes = ['COVID19', 'NORMAL', 'PNEUMONIA']

# Contar imagens por classe nos conjuntos de treino e teste
train_counts = {}
test_counts = {}

for cls in classes:
    train_dir = os.path.join(TRAIN_PATH, cls)
    test_dir = os.path.join(TEST_PATH, cls)
    
    train_count = count_images_in_directory(train_dir)
    test_count = count_images_in_directory(test_dir)
    
    train_counts[cls] = train_count
    test_counts[cls] = test_count
    
    print(f"Classe {cls}:")
    print(f"  - Treino: {train_count} imagens")
    print(f"  - Teste: {test_count} imagens")

# Total de imagens
total_train = sum(train_counts.values())
total_test = sum(test_counts.values())
total_images = total_train + total_test

print(f"\nTotal de imagens:")
print(f"  - Treino: {total_train}")
print(f"  - Teste: {total_test}")
print(f"  - Total: {total_images}")

# Criar DataFrames para visualização
train_df = pd.DataFrame(list(train_counts.items()), columns=['Classe', 'Contagem'])
train_df['Conjunto'] = 'Treino'

test_df = pd.DataFrame(list(test_counts.items()), columns=['Classe', 'Contagem'])
test_df['Conjunto'] = 'Teste'

# Combinar os DataFrames
data_df = pd.concat([train_df, test_df], ignore_index=True)

# Verificar desbalanceamento de classes
print("\nDistribuição percentual no conjunto de treino:")
for cls in classes:
    percentage = (train_counts[cls] / total_train) * 100
    print(f"  - {cls}: {percentage:.2f}%")

print("\nDistribuição percentual no conjunto de teste:")
for cls in classes:
    percentage = (test_counts[cls] / total_test) * 100
    print(f"  - {cls}: {percentage:.2f}%")

In [ ]:
# Visualizar distribuição de classes
plt.figure(figsize=(14, 6))

# Gráfico de barras para o conjunto de treino
plt.subplot(1, 2, 1)
sns.barplot(x='Classe', y='Contagem', data=train_df, palette='viridis')
plt.title('Distribuição de Classes no Conjunto de Treino')
plt.ylabel('Número de Imagens')
plt.grid(True, alpha=0.3)

# Gráfico de barras para o conjunto de teste
plt.subplot(1, 2, 2)
sns.barplot(x='Classe', y='Contagem', data=test_df, palette='viridis')
plt.title('Distribuição de Classes no Conjunto de Teste')
plt.ylabel('Número de Imagens')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Gráfico de pizza para visualizar a proporção de cada classe no conjunto de treino
plt.figure(figsize=(10, 5))

# Treino
plt.subplot(1, 2, 1)
plt.pie([train_counts[cls] for cls in classes], labels=classes, autopct='%1.1f%%', startangle=90, shadow=True)
plt.title('Proporção de Classes no Conjunto de Treino')

# Teste
plt.subplot(1, 2, 2)
plt.pie([test_counts[cls] for cls in classes], labels=classes, autopct='%1.1f%%', startangle=90, shadow=True)
plt.title('Proporção de Classes no Conjunto de Teste')

plt.tight_layout()
plt.show()

## 3. Análise das Características das Imagens

Vamos examinar as características das imagens, como dimensões (altura e largura), formato (RGB ou escala de cinza), e bits por pixel. Isso ajudará a definir o pré-processamento adequado para o treinamento das redes neurais.

In [ ]:
# Função para carregar a imagem usando a função recomendada na especificação
def load_img(path):
    # Le a imagem em diversos formatos e garante que a imagem tenha 3 canais
    img = Image.open(path).convert('RGB')
    # converte para um tensor do pytorch
    img = v2.functional.to_image(img)
    # garante que seja uma imagem de 8 bits reescalando os valores adequadamente
    img = v2.functional.to_dtype(img, dtype=torch.uint8, scale=True)
    return img

# Função para analisar características de uma amostra de imagens
def analyze_image_characteristics(directory, sample_size=20):
    image_files = []
    
    # Coletar todos os arquivos de imagem no diretório
    for cls in classes:
        class_dir = os.path.join(directory, cls)
        if os.path.exists(class_dir):
            files = [os.path.join(class_dir, f) for f in os.listdir(class_dir) 
                     if os.path.isfile(os.path.join(class_dir, f)) and 
                     (f.endswith('.jpg') or f.endswith('.jpeg') or f.endswith('.png'))]
            image_files.extend([(f, cls) for f in files])
    
    # Amostrar aleatoriamente
    if len(image_files) > sample_size:
        image_files = random.sample(image_files, sample_size)
    
    # Características para análise
    widths = []
    heights = []
    aspect_ratios = []
    channels = []
    formats = []
    bit_depths = []
    file_sizes = []
    classes_list = []
    
    # Analisar cada imagem
    for file_path, cls in tqdm(image_files, desc="Analisando imagens"):
        try:
            # Informações básicas do arquivo
            file_size = os.path.getsize(file_path) / 1024  # em KB
            file_sizes.append(file_size)
            
            # Abrir imagem com PIL
            with Image.open(file_path) as img:
                width, height = img.size
                format_img = img.format
                mode = img.mode
                
                widths.append(width)
                heights.append(height)
                aspect_ratios.append(width / height)
                formats.append(format_img)
                
                # Determinar número de canais
                if mode == 'RGB':
                    channels.append(3)
                elif mode == 'RGBA':
                    channels.append(4)
                elif mode == 'L':
                    channels.append(1)
                else:
                    channels.append(None)
                
                # Determinar bits por pixel
                if hasattr(img, 'bits'):
                    bit_depths.append(img.bits)
                else:
                    if mode in ['RGB', 'RGBA', 'L']:
                        bit_depths.append(8)  # Default para muitos formatos
                    else:
                        bit_depths.append(None)
            
            classes_list.append(cls)
        except Exception as e:
            print(f"Erro ao analisar {file_path}: {e}")
    
    # Criar DataFrame com as características
    data = {
        'Classe': classes_list,
        'Largura': widths,
        'Altura': heights,
        'Proporção': aspect_ratios,
        'Canais': channels,
        'Formato': formats,
        'Bits': bit_depths,
        'Tamanho (KB)': file_sizes
    }
    
    return pd.DataFrame(data)

# Analisar imagens do conjunto de treino
print("Analisando imagens do conjunto de treino...")
train_images_df = analyze_image_characteristics(TRAIN_PATH, sample_size=50)

# Exibir estatísticas das imagens
print("\nEstatísticas das imagens de treino:")
print(train_images_df.describe())

# Exibir informações específicas por classe
print("\nEstatísticas por classe:")
print(train_images_df.groupby('Classe').agg({
    'Largura': ['mean', 'min', 'max'],
    'Altura': ['mean', 'min', 'max'],
    'Proporção': ['mean', 'min', 'max'],
    'Canais': pd.Series.mode,
    'Bits': pd.Series.mode,
    'Tamanho (KB)': ['mean', 'min', 'max']
}))

In [ ]:
# Visualizar a distribuição de dimensões das imagens
plt.figure(figsize=(16, 12))

# Histograma das larguras
plt.subplot(2, 2, 1)
sns.histplot(train_images_df['Largura'], bins=20, kde=True)
plt.title('Distribuição de Larguras')
plt.xlabel('Largura (pixels)')
plt.grid(True, alpha=0.3)

# Histograma das alturas
plt.subplot(2, 2, 2)
sns.histplot(train_images_df['Altura'], bins=20, kde=True)
plt.title('Distribuição de Alturas')
plt.xlabel('Altura (pixels)')
plt.grid(True, alpha=0.3)

# Histograma das proporções
plt.subplot(2, 2, 3)
sns.histplot(train_images_df['Proporção'], bins=20, kde=True)
plt.title('Distribuição de Proporções (Largura/Altura)')
plt.xlabel('Proporção')
plt.grid(True, alpha=0.3)

# Histograma dos tamanhos de arquivo
plt.subplot(2, 2, 4)
sns.histplot(train_images_df['Tamanho (KB)'], bins=20, kde=True)
plt.title('Distribuição de Tamanhos de Arquivo')
plt.xlabel('Tamanho (KB)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Visualizar a relação entre largura e altura
plt.figure(figsize=(10, 8))
sns.scatterplot(data=train_images_df, x='Largura', y='Altura', hue='Classe', alpha=0.7)
plt.title('Relação entre Largura e Altura por Classe')
plt.grid(True, alpha=0.3)
plt.show()

# Contagem de canais e formatos
plt.figure(figsize=(14, 6))

# Contagem de canais
plt.subplot(1, 2, 1)
sns.countplot(x='Canais', data=train_images_df, palette='viridis')
plt.title('Distribuição de Canais')
plt.xlabel('Número de Canais')
plt.ylabel('Contagem')
plt.grid(True, alpha=0.3)

# Contagem de formatos
plt.subplot(1, 2, 2)
sns.countplot(x='Formato', data=train_images_df, palette='viridis')
plt.title('Distribuição de Formatos')
plt.xlabel('Formato')
plt.ylabel('Contagem')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Boxplot das dimensões por classe
plt.figure(figsize=(14, 6))

# Boxplot das larguras por classe
plt.subplot(1, 2, 1)
sns.boxplot(x='Classe', y='Largura', data=train_images_df, palette='viridis')
plt.title('Distribuição de Larguras por Classe')
plt.ylabel('Largura (pixels)')
plt.grid(True, alpha=0.3)

# Boxplot das alturas por classe
plt.subplot(1, 2, 2)
sns.boxplot(x='Classe', y='Altura', data=train_images_df, palette='viridis')
plt.title('Distribuição de Alturas por Classe')
plt.ylabel('Altura (pixels)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Visualização de Exemplos de Imagens por Classe

Vamos visualizar exemplos de imagens de cada classe (COVID-19, Pneumonia, Normal) para entender melhor as características visuais que as redes neurais precisarão aprender.

In [ ]:
# Função para obter exemplos de imagens de cada classe
def get_example_images(directory, n_examples=3):
    examples = {}
    
    for cls in classes:
        class_dir = os.path.join(directory, cls)
        if os.path.exists(class_dir):
            files = [os.path.join(class_dir, f) for f in os.listdir(class_dir) 
                     if os.path.isfile(os.path.join(class_dir, f)) and 
                     (f.endswith('.jpg') or f.endswith('.jpeg') or f.endswith('.png'))]
            
            if len(files) > n_examples:
                files = random.sample(files, n_examples)
            
            examples[cls] = files
    
    return examples

# Obter exemplos de imagens do conjunto de treino
print("Obtendo exemplos de imagens...")
train_examples = get_example_images(TRAIN_PATH, n_examples=4)

# Função para mostrar exemplos de imagens
def show_examples(examples):
    n_classes = len(classes)
    n_examples = len(examples[classes[0]])
    
    plt.figure(figsize=(15, 5*n_classes))
    
    for i, cls in enumerate(classes):
        for j, img_path in enumerate(examples[cls]):
            plt.subplot(n_classes, n_examples, i*n_examples + j + 1)
            
            try:
                # Carregar imagem com a função recomendada
                img_tensor = load_img(img_path)
                img_np = img_tensor.permute(1, 2, 0).numpy()
                
                plt.imshow(img_np)
                plt.title(f'{cls} - Exemplo {j+1}')
                plt.axis('off')
                
                # Mostrar dimensões e formato
                img_pil = Image.open(img_path)
                plt.xlabel(f'{img_pil.size[0]}x{img_pil.size[1]}, {img_pil.mode}')
                
            except Exception as e:
                plt.text(0.5, 0.5, f"Erro: {str(e)}", ha='center', va='center')
    
    plt.tight_layout()
    plt.show()

# Mostrar exemplos de imagens
show_examples(train_examples)

## 5. Visualização de Pré-processamento e Data Augmentation

Vamos testar algumas transformações de pré-processamento e data augmentation que poderão ser utilizadas no treinamento das redes neurais.

In [ ]:
# Selecionar uma imagem para testar as transformações
sample_cls = random.choice(classes)
sample_img_path = train_examples[sample_cls][0]
print(f"Imagem selecionada: {sample_img_path}")

# Carregar a imagem
sample_img_tensor = load_img(sample_img_path)
sample_img_np = sample_img_tensor.permute(1, 2, 0).numpy()

# Definir transformações de pré-processamento básicas
preprocess_transforms = v2.Compose([
    v2.Resize((224, 224)),  # Redimensionar para 224x224 (comum para muitas CNNs)
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True)  # Normalizar para [0, 1]
])

# Aplicar pré-processamento básico
preprocessed_img = preprocess_transforms(sample_img_tensor)
preprocessed_img_np = preprocessed_img.permute(1, 2, 0).numpy()

# Definir transformações de data augmentation
augment_transforms = v2.Compose([
    v2.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomRotation(degrees=15),
    v2.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    v2.ColorJitter(brightness=0.2, contrast=0.2),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True)
])

# Aplicar data augmentation várias vezes
n_augmentations = 8
augmented_imgs = [augment_transforms(sample_img_tensor) for _ in range(n_augmentations)]
augmented_imgs_np = [img.permute(1, 2, 0).numpy() for img in augmented_imgs]

# Visualizar imagem original, pré-processada e aumentada
plt.figure(figsize=(15, 10))

# Imagem original
plt.subplot(2, 5, 1)
plt.imshow(sample_img_np)
plt.title('Original')
plt.axis('off')

# Imagem pré-processada
plt.subplot(2, 5, 2)
plt.imshow(preprocessed_img_np)
plt.title('Pré-processada (224x224)')
plt.axis('off')

# Imagens aumentadas
for i, aug_img_np in enumerate(augmented_imgs_np):
    plt.subplot(2, 5, i+3)
    plt.imshow(aug_img_np)
    plt.title(f'Aumento #{i+1}')
    plt.axis('off')

plt.tight_layout()
plt.show()

# Definir transformações de pré-processamento normalizadas (para modelos pré-treinados)
# Usando média e desvio padrão do ImageNet
imagenet_normalize = v2.Compose([
    v2.Resize((224, 224)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Aplicar normalização ImageNet
normalized_img = imagenet_normalize(sample_img_tensor)

# Não podemos visualizar diretamente a imagem normalizada (valores fora de [0,1])
# Vamos desnormalizar
denormalize = v2.Compose([
    v2.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.225]
    )
])

denormalized_img = denormalize(normalized_img)
denormalized_img_np = denormalized_img.permute(1, 2, 0).numpy()

# Visualizar imagem original e normalizada
plt.figure(figsize=(10, 5))

# Imagem original
plt.subplot(1, 2, 1)
plt.imshow(sample_img_np)
plt.title('Original')
plt.axis('off')

# Imagem normalizada e depois desnormalizada
plt.subplot(1, 2, 2)
plt.imshow(np.clip(denormalized_img_np, 0, 1))  # Clip para garantir valores no intervalo [0,1]
plt.title('Normalizado -> Desnormalizado')
plt.axis('off')

plt.tight_layout()
plt.show()

## 6. Conclusões e Recomendações

Com base na análise exploratória realizada, podemos extrair as seguintes conclusões e recomendações para o treinamento das redes neurais:

### Principais Insights:

1. **Distribuição de Classes:**
   - O dataset pode apresentar desbalanceamento entre as classes, o que pode impactar o treinamento.
   - Pode ser necessário utilizar técnicas como amostragem ponderada ou ajuste nos pesos da função de perda.

2. **Características das Imagens:**
   - As imagens possuem dimensões variadas, sendo necessário padronizar para um tamanho fixo.
   - Algumas imagens são em escala de cinza e outras em RGB, sendo importante tratar essa heterogeneidade.
   - A qualidade das imagens varia, com diferentes resoluções e formatos.

3. **Pré-processamento Recomendado:**
   - Redimensionar todas as imagens para 224x224 (padrão para muitas CNNs pré-treinadas).
   - Converter todas as imagens para RGB (3 canais).
   - Normalizar os valores de pixel usando as estatísticas do ImageNet para modelos pré-treinados.

4. **Data Augmentation Recomendado:**
   - Rotações leves (±15 graus) - apropriado para raios-X que mantêm seu significado mesmo com pequenas rotações.
   - Pequenas translações (10% em cada direção).
   - Ajustes de brilho e contraste (20% de variação) - simula diferentes condições de exposição dos raios-X.
   - Cortes aleatórios preservando a escala - ajuda a focar em diferentes partes da imagem.
   - Evitar espelhamento vertical, pois pode alterar características anatômicas importantes.

5. **Considerações para o Treinamento:**
   - Dividir adequadamente o conjunto de treino em treino e validação (por exemplo, 80/20).
   - Monitorar cuidadosamente o overfitting, especialmente devido ao número limitado de amostras.
   - Utilizar callbacks como Early Stopping para evitar overfitting.
   - Considerar utilizar batch normalization nas redes customizadas para melhorar o treinamento.

6. **Modelos Pré-treinados Recomendados:**
   - DenseNet ou ResNet podem ser boas opções devido ao seu desempenho em tarefas médicas.
   - MobileNetV2/V3 para aplicações que necessitam de menor carga computacional.

### Próximos Passos:

1. Implementar os pipelines de dados com as transformações adequadas.
2. Desenvolver a CNN customizada com base nas características identificadas.
3. Adaptar modelos pré-treinados para a tarefa específica.
4. Treinar os modelos monitorando as métricas de desempenho.
5. Realizar análise comparativa entre os diferentes modelos.